In [ ]:
import json
from itertools import product
from typing import List, Dict, Any, Optional
from collections import defaultdict
from pathlib import Path
import sys
import pandas as pd
import csv

sys.path.append(str(Path.cwd().parent / "src"))
from utils.config_loader import ConfigLoader
from core.cohort_generator import CohortGenerator
from core.deduplicator import InsightDeduplicator

pd.set_option("display.max_columns", None)

: 

# Generating embeddings

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import numpy as np
model = SentenceTransformer("all-MiniLM-L6-v2")

d:\genie\dyk\dyk\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [170]:
"""
Deduplication Analysis Module

Main class for analyzing insight duplication with configurable metrics.
Can be used programmatically or via CLI.
"""

import json
import numpy as np
import pandas as pd
from pathlib import Path
from typing import List, Dict, Optional, Tuple
from collections import Counter
import pickle

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx


class DeduplicationAnalyzer:
    """
    Analyzes insight duplication with multiple metrics and visualizations.
    
    Example:
        >>> analyzer = DeduplicationAnalyzer(insights, threshold=0.85)
        >>> analyzer.compute_embeddings()
        >>> results = analyzer.analyze()
        >>> model_comparison = analyzer.get_model_comparison()
    """
    
    def __init__(
        self,
        insights: List[Dict],
        weights: Optional[Dict[str, float]] = None,
        threshold: Optional[float] = None,
        model_name: str = 'all-MiniLM-L6-v2',
        n_greedy_runs: int = 10,
        random_seed: int = 42
    ):
        """
        Initialize DeduplicationAnalyzer
        
        Args:
            insights: List of insight dictionaries with 'hook', 'explanation', 'action'
            weights: Component weights, default {'hook': 0.4, 'action': 0.4, 'explanation': 0.2}
            threshold: Similarity threshold for duplicates (if None, must tune or set later)
            model_name: Sentence transformer model name
            n_greedy_runs: Number of runs for greedy deduplication averaging
            random_seed: Random seed for reproducibility
        """
        self.insights = insights
        self.weights = weights or {'hook': 0.4, 'action': 0.4, 'explanation': 0.2}
        self.threshold = threshold
        self.model_name = model_name
        self.n_greedy_runs = n_greedy_runs
        self.random_seed = random_seed
        
        # Validate weights
        total_weight = sum(self.weights.values())
        if not np.isclose(total_weight, 1.0):
            raise ValueError(f"Weights must sum to 1.0, got {total_weight}")
        
        # Internal state
        self._model = None
        self._embeddings = None
        self._similarity_matrix = None
        self._metadata = None
        self._results = None
    
    @property
    def model(self) -> SentenceTransformer:
        """Lazy load the sentence transformer model"""
        if self._model is None:
            print(f"Loading model: {self.model_name}...")
            self._model = SentenceTransformer(self.model_name)
        return self._model
    
    def compute_embeddings(self, show_progress: bool = True) -> 'DeduplicationAnalyzer':
        """
        Generate embeddings for all insights
        
        Args:
            show_progress: Show progress bar during encoding
            
        Returns:
            self (for method chaining)
        """
        print(f"Generating embeddings for {len(self.insights)} insights...")
        
        # Extract text components
        full_texts = [
            f"{ins.get('hook', '')} {ins.get('explanation', '')} {ins.get('action', '')}"
            for ins in self.insights
        ]
        hooks = [ins.get('hook', '') for ins in self.insights]
        explanations = [ins.get('explanation', '') for ins in self.insights]
        actions = [ins.get('action', '') for ins in self.insights]
        
        # Generate embeddings
        self._embeddings = {
            'full': self.model.encode(full_texts, show_progress_bar=show_progress, batch_size=32),
            'hook': self.model.encode(hooks, show_progress_bar=show_progress, batch_size=32),
            'explanation': self.model.encode(explanations, show_progress_bar=show_progress, batch_size=32),
            'action': self.model.encode(actions, show_progress_bar=show_progress, batch_size=32)
        }
        
        print(f"✓ Embeddings generated: {self._embeddings['full'].shape}")
        return self
    
    def _compute_weighted_similarity(self) -> np.ndarray:
        """Compute weighted similarity matrix"""
        if self._embeddings is None:
            raise ValueError("Must call compute_embeddings() first")
        
        # Compute similarity for each component
        hook_sim = cosine_similarity(self._embeddings['hook'])
        explanation_sim = cosine_similarity(self._embeddings['explanation'])
        action_sim = cosine_similarity(self._embeddings['action'])
        
        # Weighted combination
        weighted_sim = (
            self.weights['hook'] * hook_sim +
            self.weights['explanation'] * explanation_sim +
            self.weights['action'] * action_sim
        )
        
        return weighted_sim
    
    def get_similarity_matrix(self) -> np.ndarray:
        """
        Get or compute the weighted similarity matrix
        
        Returns:
            Similarity matrix (n x n)
        """
        if self._similarity_matrix is None:
            print("Computing weighted similarity matrix...")
            print(f"Weights: Hook={self.weights['hook']}, "
                  f"Action={self.weights['action']}, "
                  f"Explanation={self.weights['explanation']}")
            self._similarity_matrix = self._compute_weighted_similarity()
        return self._similarity_matrix
    
    def _extract_metadata(self) -> List[Dict]:
        """Extract model, cohort, template metadata from insights"""
        if self._metadata is None:
            self._metadata = []
            for insight in self.insights:
                meta = insight.get('metadata', {})
                self._metadata.append({
                    'model': meta.get('generation_model', 'unknown'),
                    'cohort': meta.get('cohort', {}).get('description', 'unknown'),
                    'template': meta.get('insight_template', {}).get('type', 'unknown')
                })
        return self._metadata
    
    def _greedy_deduplication(
        self, 
        similarity_matrix: np.ndarray,
        threshold: float
    ) -> Tuple[float, float, List[int]]:
        """
        Greedy deduplication with multiple runs
        
        Returns:
            (mean unique count, std, list of all run results)
        """
        np.random.seed(self.random_seed)
        n = similarity_matrix.shape[0]
        results = []
        
        for run in range(self.n_greedy_runs):
            order = np.random.permutation(n)
            kept_indices = []
            
            for idx in order:
                is_duplicate = False
                for kept_idx in kept_indices:
                    if similarity_matrix[idx, kept_idx] >= threshold:
                        is_duplicate = True
                        break
                
                if not is_duplicate:
                    kept_indices.append(idx)
            
            results.append(len(kept_indices))
        
        return np.mean(results), np.std(results), results
    
    def _connected_components_clustering(
        self,
        similarity_matrix: np.ndarray,
        threshold: float
    ) -> Tuple[int, List[int]]:
        """
        Use connected components to find unique clusters
        
        Returns:
            (number of clusters, cluster sizes)
        """
        n = similarity_matrix.shape[0]
        
        # Build graph
        G = nx.Graph()
        G.add_nodes_from(range(n))
        
        # Add edges for similar pairs
        for i in range(n):
            for j in range(i + 1, n):
                if similarity_matrix[i, j] >= threshold:
                    G.add_edge(i, j)
        
        # Find connected components
        clusters = list(nx.connected_components(G))
        cluster_sizes = [len(c) for c in clusters]
        
        return len(clusters), cluster_sizes
    
    def _compute_duplicate_counts(
        self,
        similarity_matrix: np.ndarray,
        threshold: float
    ) -> np.ndarray:
        """
        For each insight, count how many duplicates it has
        
        Returns:
            Array of duplicate counts per insight
        """
        n = similarity_matrix.shape[0]
        duplicate_counts = np.zeros(n, dtype=int)
        
        for i in range(n):
            similar_mask = similarity_matrix[i] >= threshold
            duplicate_counts[i] = similar_mask.sum() - 1  # Subtract self
        
        return duplicate_counts
    
    def _analyze_group(
        self,
        similarity_matrix: np.ndarray,
        threshold: float,
        metadata: List[Dict],
        group_by: str
    ) -> pd.DataFrame:
        """
        Analyze duplication metrics grouped by model/cohort/template
        
        Args:
            similarity_matrix: Similarity matrix
            threshold: Similarity threshold
            metadata: List of metadata dicts
            group_by: 'model', 'cohort', or 'template'
            
        Returns:
            DataFrame with metrics per group
        """
        df = pd.DataFrame(metadata)
        df['idx'] = range(len(metadata))
        
        results = []
        
        for group_value in df[group_by].unique():
            group_indices = df[df[group_by] == group_value]['idx'].values
            n_insights = len(group_indices)
            
            if n_insights == 0:
                continue
            
            # Extract submatrix for this group
            submatrix = similarity_matrix[np.ix_(group_indices, group_indices)]
            
            # Compute metrics
            greedy_mean, greedy_std, _ = self._greedy_deduplication(submatrix, threshold)
            n_clusters, cluster_sizes = self._connected_components_clustering(submatrix, threshold)
            duplicate_counts = self._compute_duplicate_counts(submatrix, threshold)
            
            results.append({
                group_by: group_value,
                'total_insights': n_insights,
                'greedy_unique_mean': greedy_mean,
                'greedy_unique_std': greedy_std,
                'greedy_unique_pct': (greedy_mean / n_insights * 100),
                'cluster_count': n_clusters,
                'cluster_pct': (n_clusters / n_insights * 100),
                'mean_duplicates_per_insight': duplicate_counts.mean(),
                'max_duplicates': duplicate_counts.max(),
                'pct_with_0_duplicates': (duplicate_counts == 0).sum() / n_insights * 100,
                'pct_with_5plus_duplicates': (duplicate_counts >= 5).sum() / n_insights * 100
            })
        
        return pd.DataFrame(results).sort_values('greedy_unique_pct', ascending=False)
    
    def _analyze_overlap(
        self,
        similarity_matrix: np.ndarray,
        threshold: float,
        metadata: List[Dict],
        group_by: str
    ) -> pd.DataFrame:
        """
        Analyze cross-group overlap (pairwise)
        
        Args:
            similarity_matrix: Similarity matrix
            threshold: Similarity threshold
            metadata: List of metadata dicts
            group_by: 'cohort' or 'template'
            
        Returns:
            DataFrame with pairwise overlap metrics
        """
        df = pd.DataFrame(metadata)
        df['idx'] = range(len(metadata))
        
        groups = df[group_by].unique()
        results = []
        
        for i, group1 in enumerate(groups):
            for group2 in groups[i+1:]:
                indices1 = df[df[group_by] == group1]['idx'].values
                indices2 = df[df[group_by] == group2]['idx'].values
                
                # Get cross-similarity submatrix
                cross_sim = similarity_matrix[np.ix_(indices1, indices2)]
                
                # Count overlaps
                n_overlaps = (cross_sim >= threshold).sum()
                total_pairs = len(indices1) * len(indices2)
                
                results.append({
                    f'{group_by}_1': group1,
                    f'{group_by}_2': group2,
                    'overlap_count': n_overlaps,
                    'total_possible_pairs': total_pairs,
                    'overlap_pct': (n_overlaps / total_pairs * 100) if total_pairs > 0 else 0
                })
        
        return pd.DataFrame(results).sort_values('overlap_pct', ascending=False)
    
    def _find_worst_insights(
        self,
        similarity_matrix: np.ndarray,
        threshold: float,
        top_n: int = 20
    ) -> pd.DataFrame:
        """Find the most duplicated insights"""
        duplicate_counts = self._compute_duplicate_counts(similarity_matrix, threshold)
        worst_indices = np.argsort(duplicate_counts)[::-1][:top_n]
        
        results = []
        for rank, idx in enumerate(worst_indices, 1):
            insight = self.insights[idx]
            dup_count = duplicate_counts[idx]
            
            results.append({
                'rank': rank,
                'duplicate_count': int(dup_count),
                'hook': insight.get('hook', ''),
                'explanation': insight.get('explanation', ''),
                'action': insight.get('action', ''),
                'model': insight.get('metadata', {}).get('generation_model', 'unknown'),
                'cohort': insight.get('metadata', {}).get('cohort', {}).get('description', 'unknown'),
                'template': insight.get('metadata', {}).get('insight_template', {}).get('type', 'unknown')
            })
        
        return pd.DataFrame(results)
    
    def analyze(self, threshold: Optional[float] = None) -> Dict[str, pd.DataFrame]:
        """
        Run complete deduplication analysis
        
        Args:
            threshold: Similarity threshold (uses self.threshold if not provided)
            
        Returns:
            Dictionary with all analysis results:
                - 'overall': Overall statistics
                - 'by_model': Model comparison
                - 'by_cohort': Cohort analysis
                - 'by_template': Template analysis
                - 'template_overlap': Cross-template overlap
                - 'cohort_overlap': Cross-cohort overlap
                - 'worst_insights': Most duplicated insights
        """
        if threshold is not None:
            self.threshold = threshold
        
        if self.threshold is None:
            raise ValueError("Threshold must be set. Call tune_threshold() or pass threshold parameter.")
        
        print(f"\nRunning analysis with threshold={self.threshold}...")
        
        # Get similarity matrix
        similarity_matrix = self.get_similarity_matrix()
        
        # Extract metadata
        metadata = self._extract_metadata()
        
        # Overall statistics
        print("Computing overall statistics...")
        greedy_mean, greedy_std, _ = self._greedy_deduplication(similarity_matrix, self.threshold)
        n_clusters, _ = self._connected_components_clustering(similarity_matrix, self.threshold)
        duplicate_counts = self._compute_duplicate_counts(similarity_matrix, self.threshold)
        
        overall_stats = {
            'total_insights': len(self.insights),
            'greedy_unique_mean': greedy_mean,
            'greedy_unique_std': greedy_std,
            'greedy_unique_pct': (greedy_mean / len(self.insights) * 100),
            'cluster_count': n_clusters,
            'cluster_pct': (n_clusters / len(self.insights) * 100),
            'mean_duplicates_per_insight': duplicate_counts.mean()
        }
        
        # Group analyses
        print("Analyzing by model...")
        by_model = self._analyze_group(similarity_matrix, self.threshold, metadata, 'model')
        
        print("Analyzing by cohort...")
        by_cohort = self._analyze_group(similarity_matrix, self.threshold, metadata, 'cohort')
        
        print("Analyzing by template...")
        by_template = self._analyze_group(similarity_matrix, self.threshold, metadata, 'template')
        
        # Overlap analyses
        print("Analyzing cross-template overlap...")
        template_overlap = self._analyze_overlap(similarity_matrix, self.threshold, metadata, 'template')
        
        print("Analyzing cross-cohort overlap...")
        cohort_overlap = self._analyze_overlap(similarity_matrix, self.threshold, metadata, 'cohort')
        
        # Worst insights
        print("Finding worst-performing insights...")
        worst_insights = self._find_worst_insights(similarity_matrix, self.threshold)
        
        self._results = {
            'overall': pd.DataFrame([overall_stats]),
            'by_model': by_model,
            'by_cohort': by_cohort,
            'by_template': by_template,
            'template_overlap': template_overlap,
            'cohort_overlap': cohort_overlap,
            'worst_insights': worst_insights,
            'duplicate_counts': duplicate_counts  # For distribution analysis
        }
        
        print("✓ Analysis complete!")
        return self._results
    
    def get_results(self) -> Dict[str, pd.DataFrame]:
        """Get analysis results (must call analyze() first)"""
        if self._results is None:
            raise ValueError("No results available. Call analyze() first.")
        return self._results
    
    def get_model_comparison(self) -> pd.DataFrame:
        """Get model comparison metrics"""
        return self.get_results()['by_model']
    
    def get_worst_insights(self, top_n: int = 20) -> pd.DataFrame:
        """Get most duplicated insights"""
        return self.get_results()['worst_insights'].head(top_n)
    
    def get_best_model(self) -> str:
        """Get the best performing model name"""
        model_df = self.get_model_comparison()
        best_idx = model_df['greedy_unique_pct'].idxmax()
        return model_df.loc[best_idx, 'model']
    
    def save_embeddings(self, filepath: str) -> None:
        """Save computed embeddings to disk"""
        if self._embeddings is None:
            raise ValueError("No embeddings to save. Call compute_embeddings() first.")
        
        with open(filepath, 'wb') as f:
            pickle.dump({
                'embeddings': self._embeddings,
                'insights': self.insights,
                'weights': self.weights
            }, f)
        print(f"✓ Embeddings saved to {filepath}")
    
    def load_embeddings(self, filepath: str) -> 'DeduplicationAnalyzer':
        """Load pre-computed embeddings from disk"""
        with open(filepath, 'rb') as f:
            data = pickle.load(f)
        
        self._embeddings = data['embeddings']
        print(f"✓ Embeddings loaded from {filepath}")
        return self
    
    def __repr__(self):
        return (f"DeduplicationAnalyzer(n_insights={len(self.insights)}, "
                f"threshold={self.threshold}, weights={self.weights})")

In [ ]:
with open("../output/deepseek/insights_final.json", "r") as f:
    data = json.load(f)
insights = data["insights"]

analyzer = DeduplicationAnalyzer(insights,
                                 weights=None,
                                 threshold = 0.85)
analyzer.compute_embeddings()
results = analyzer.analyze(threshold=0.85)
results["overall"]

,total_insights,greedy_unique_mean,greedy_unique_std,greedy_unique_pct,cluster_count,cluster_pct,mean_duplicates_per_insight
0,410,386.1,1.3,94.170732,382,93.170732,0.2


In [174]:
with open("../output/gemini_flash_2.5/insights_final.json", "r") as f:
    data = json.load(f)
insights = data["insights"]

analyzer = DeduplicationAnalyzer(insights,
                                 weights=None,
                                 threshold = 0.85)
analyzer.compute_embeddings(show_progress=False)
results = analyzer.analyze(threshold=0.85)
results["overall"]

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2


Generating embeddings for 425 insights...
Loading model: all-MiniLM-L6-v2...
✓ Embeddings generated: (425, 384)

Running analysis with threshold=0.85...
Computing weighted similarity matrix...
Weights: Hook=0.4, Action=0.4, Explanation=0.2
Computing overall statistics...
Analyzing by model...
Analyzing by cohort...
Analyzing by template...
Analyzing cross-template overlap...
Analyzing cross-cohort overlap...
Finding worst-performing insights...
✓ Analysis complete!


,total_insights,greedy_unique_mean,greedy_unique_std,greedy_unique_pct,cluster_count,cluster_pct,mean_duplicates_per_insight
0,425,383.1,1.3,90.141176,375,88.235294,0.324706


In [193]:
with open("../output/grok-fast/insights_final.json", "r") as f:
    data = json.load(f)
insights = data["insights"]

analyzer = DeduplicationAnalyzer(insights,
                                 weights={'hook': 0.1, 'action': 0.8, 'explanation': 0.1},
                                 threshold = 0.85)
analyzer.compute_embeddings(show_progress=False)
results = analyzer.analyze(threshold=0.85)
results["overall"]

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2


Generating embeddings for 295 insights...
Loading model: all-MiniLM-L6-v2...
✓ Embeddings generated: (295, 384)

Running analysis with threshold=0.85...
Computing weighted similarity matrix...
Weights: Hook=0.1, Action=0.8, Explanation=0.1
Computing overall statistics...
Analyzing by model...
Analyzing by cohort...
Analyzing by template...
Analyzing cross-template overlap...
Analyzing cross-cohort overlap...
Finding worst-performing insights...
✓ Analysis complete!


,total_insights,greedy_unique_mean,greedy_unique_std,greedy_unique_pct,cluster_count,cluster_pct,mean_duplicates_per_insight
0,295,275.0,1.264911,93.220339,271,91.864407,0.250847


In [188]:
"""
Threshold Tuning Module

Interactive threshold tuning through manual review of insight pairs.
"""

import numpy as np
import random
from typing import List, Dict, Tuple


class ThresholdTuner:
    """
    Interactive threshold tuner for similarity analysis.
    
    Samples insight pairs at different similarity ranges for manual review.
    """
    
    def __init__(self, similarity_matrix: np.ndarray, insights: List[Dict]):
        """
        Initialize ThresholdTuner
        
        Args:
            similarity_matrix: Precomputed similarity matrix (n x n)
            insights: List of insight dictionaries
        """
        self.similarity_matrix = similarity_matrix
        self.insights = insights
    
    def sample_pairs(
        self,
        samples_per_range: int = 5,
        ranges: List[Tuple[float, float, str]] = None
    ) -> Dict[str, List[Tuple[int, int, float]]]:
        """
        Sample insight pairs at different similarity ranges
        
        Args:
            samples_per_range: Number of pairs to sample per range
            ranges: List of (min_sim, max_sim, label) tuples
            
        Returns:
            Dict mapping range labels to lists of (idx1, idx2, similarity) tuples
        """
        if ranges is None:
            ranges = [
                (0.95, 1.00, "Very High"),
                (0.90, 0.95, "High"),
                (0.85, 0.90, "Medium-High"),
                (0.80, 0.85, "Medium"),
                (0.75, 0.80, "Medium-Low"),
                (0.70, 0.75, "Low")
            ]
        
        n = len(self.insights)
        sampled_pairs = {label: [] for _, _, label in ranges}
        
        # Get upper triangle indices (avoid diagonal and duplicates)
        upper_tri_i, upper_tri_j = np.triu_indices(n, k=1)
        total_pairs = len(upper_tri_i)
        
        print(f"Total unique pairs: {total_pairs:,}")
        
        # Sample from each range
        for min_sim, max_sim, label in ranges:
            # Find pairs in this similarity range using vectorized operations
            similarities = self.similarity_matrix[upper_tri_i, upper_tri_j]
            mask = (similarities >= min_sim) & (similarities < max_sim)
            
            # Get indices of pairs in range
            indices_in_range = np.where(mask)[0]
            count_in_range = len(indices_in_range)
            
            if count_in_range == 0:
                print(f"\n⚠️  No pairs found in range {label} ({min_sim:.2f}-{max_sim:.2f})")
                continue
            
            # Sample randomly
            sample_size = min(samples_per_range, count_in_range)
            sampled_indices = np.random.choice(indices_in_range, size=sample_size, replace=False)
            
            # Store sampled pairs
            sampled = [
                (int(upper_tri_i[idx]), int(upper_tri_j[idx]), float(similarities[idx]))
                for idx in sampled_indices
            ]
            sampled_pairs[label] = sampled
            
            print(f"{label} ({min_sim:.2f}-{max_sim:.2f}): {count_in_range:,} pairs, sampled {sample_size}")
        
        return sampled_pairs
    
    def display_pair(
        self,
        idx1: int,
        idx2: int,
        similarity: float,
        pair_num: int,
        total: int
    ) -> None:
        """Display a pair of insights for comparison"""
        insight1 = self.insights[idx1]
        insight2 = self.insights[idx2]
        
        print("\n" + "="*80)
        print(f"PAIR {pair_num}/{total} - Similarity: {similarity:.4f}")
        print("="*80)
        
        print("\n📌 INSIGHT 1:")
        print(f"Hook: {insight1.get('hook', 'N/A')}")
        print(f"Explanation: {insight1.get('explanation', 'N/A')}")
        print(f"Action: {insight1.get('action', 'N/A')}")
        
        # Show metadata if available
        if 'metadata' in insight1:
            cohort = insight1['metadata'].get('cohort', {}).get('cohort_id', 'N/A')
            template = insight1['metadata'].get('insight_template', {}).get('type', 'N/A')
            model = insight1['metadata'].get('generation_model', 'N/A')
            print(f"[Cohort: {cohort} | Template: {template} | Model: {model}]")
        
        print("\n📌 INSIGHT 2:")
        print(f"Hook: {insight2.get('hook', 'N/A')}")
        print(f"Explanation: {insight2.get('explanation', 'N/A')}")
        print(f"Action: {insight2.get('action', 'N/A')}")
        
        # Show metadata if available
        if 'metadata' in insight2:
            cohort = insight2['metadata'].get('cohort', {}).get('cohort_id', 'N/A')
            template = insight2['metadata'].get('insight_template', {}).get('type', 'N/A')
            model = insight2['metadata'].get('generation_model', 'N/A')
            print(f"[Cohort: {cohort} | Template: {template} | Model: {model}]")
    
    def review_samples(self, sampled_pairs: Dict[str, List[Tuple[int, int, float]]]) -> None:
        """Display sampled pairs for manual review"""
        print("\n" + "🔍 " + "="*76)
        print("SIMILARITY THRESHOLD TUNING - MANUAL REVIEW")
        print("="*78)
        print("\nReview the pairs below and decide:")
        print("- Which similarity level represents 'duplicate' insights?")
        print("- Look for where duplicates START to appear")
        print("\n")
        
        for label, pairs in sampled_pairs.items():
            if not pairs:
                continue
            
            print(f"\n{'='*80}")
            print(f"🎯 {label.upper()} SIMILARITY RANGE")
            print(f"{'='*80}")
            
            for idx, (i, j, sim) in enumerate(pairs, 1):
                self.display_pair(i, j, sim, idx, len(pairs))
        
        print("\n" + "="*80)
        print("REVIEW COMPLETE")
        print("="*80)
        print("\n💡 RECOMMENDATIONS:")
        print("- If pairs at 0.90-0.95 look like duplicates → use threshold 0.90")
        print("- If pairs at 0.85-0.90 look like duplicates → use threshold 0.85")
        print("- If pairs at 0.80-0.85 look like duplicates → use threshold 0.80")
        print("\nChoose the threshold where you START seeing duplicates.")
    
    def interactive_tune(
        self,
        samples_per_range: int = 5,
        auto_save: bool = True,
        save_path: str = 'threshold_decision.txt'
    ) -> float:
        """
        Run interactive threshold tuning session
        
        Args:
            samples_per_range: Number of pairs to show per range
            auto_save: Whether to save the chosen threshold
            save_path: Path to save threshold
            
        Returns:
            Chosen threshold value
        """
        # Sample pairs
        sampled_pairs = self.sample_pairs(samples_per_range)
        
        # Display for review
        self.review_samples(sampled_pairs)
        
        # Get user input
        print("\n" + "="*80)
        threshold_input = input("Enter your chosen threshold (e.g., 0.85): ")
        
        try:
            threshold = float(threshold_input)
            
            if auto_save:
                with open(save_path, 'w') as f:
                    f.write(f"{threshold}\n")
                print(f"\n✅ Threshold {threshold} saved to {save_path}")
            
            return threshold
            
        except ValueError:
            print("Invalid threshold. Please enter a number.")
            raise
    
    def suggest_threshold(self, target_uniqueness: float = 0.80) -> float:
        """
        Suggest a threshold based on target uniqueness percentage
        
        Args:
            target_uniqueness: Target percentage of unique insights (0.0-1.0)
            
        Returns:
            Suggested threshold value
        """
        # Simple heuristic: find threshold where ~target_uniqueness of insights are unique
        # This is approximate and may need refinement
        
        n = len(self.insights)
        upper_tri_i, upper_tri_j = np.triu_indices(n, k=1)
        similarities = self.similarity_matrix[upper_tri_i, upper_tri_j]
        
        # Try different thresholds
        thresholds = np.arange(0.70, 0.99, 0.01)
        best_threshold = 0.85
        best_diff = float('inf')
        
        for threshold in thresholds:
            # Count duplicates at this threshold
            duplicate_mask = similarities >= threshold
            n_duplicate_pairs = duplicate_mask.sum()
            
            # Rough estimate of unique percentage
            # (This is approximate, actual calculation requires greedy dedup)
            estimated_unique_pct = 1.0 - (n_duplicate_pairs / len(similarities))
            
            diff = abs(estimated_unique_pct - target_uniqueness)
            if diff < best_diff:
                best_diff = diff
                best_threshold = threshold
        
        print(f"Suggested threshold for ~{target_uniqueness:.0%} uniqueness: {best_threshold:.2f}")
        return best_threshold

In [194]:
tuner = ThresholdTuner(analyzer.get_similarity_matrix(), insights=insights)

In [195]:
sampled_pairs = tuner.sample_pairs(samples_per_range=3)

Total unique pairs: 43,365

⚠️  No pairs found in range Very High (0.95-1.00)
High (0.90-0.95): 4 pairs, sampled 3
Medium-High (0.85-0.90): 33 pairs, sampled 3
Medium (0.80-0.85): 50 pairs, sampled 3
Medium-Low (0.75-0.80): 107 pairs, sampled 3
Low (0.70-0.75): 177 pairs, sampled 3


In [196]:
tuner.review_samples(sampled_pairs)


🔍 ============================================================================
SIMILARITY THRESHOLD TUNING - MANUAL REVIEW

Review the pairs below and decide:
- Which similarity level represents 'duplicate' insights?
- Look for where duplicates START to appear



🎯 HIGH SIMILARITY RANGE

PAIR 1/3 - Similarity: 0.9090

📌 INSIGHT 1:
Hook: Did you know night shift light exposure increases hypertension risk by 40% by slashing melatonin levels 50%?
Explanation: Artificial light suppresses melatonin, disrupting circadian control of blood pressure and boosting sympathetic nervous activity in shift workers with prediabetes.
Action: Wear blue-light blocking glasses after 10pm during shifts and get 30min morning sunlight exposure daily.
[Cohort: cohort_0002 | Template: mechanism_reveal | Model: x-ai/grok-4.1-fast]

📌 INSIGHT 2:
Hook: Did you know night shift light exposure raises systolic blood pressure by 10-15 mmHg in hypertensives?
Explanation: Artificial light suppresses melatonin, activati

# Functional walkthrough

In [31]:
with open("../output/deepseek/insights_final.json", "r") as f:
    data = json.load(f)
    
insights = data["insights"]

if not insights:
    print("No insights to export to CSV")
    
csv_file = "../output/insights_final.csv"

In [32]:
try:
    with open(csv_file, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)

        # header row
        writer.writerow(
            [
                "full_insight",
                "hook",
                "explanation",
                "action",
                "source_name",
                "source_url",
                "cohort_params",
                "cohort_desc",
                "insight_template",
                "gen_model",
                "gen_temperature",
                "gen_max_tokens",
                "generated_at",
                "eval_score",
                "factual_accuracy",
                "safety",
                "faithfulness",
                "relevance",
                "actionability",
                "cultural_appropriateness",
                "recommendation",
                "revision_suggestions",
                "eval_model",
                "evaluated_at",
            ]
        )

        for idx, insight in enumerate(insights, 1):
            # Extract metadata
            metadata = insight.get("metadata", {})
            cohort = metadata.get("cohort", {})
            template = metadata.get("insight_template", {})
            eval = insight.get("evaluation", {})
            eval_result = eval.get("result", {})

            writer.writerow(
                [
                    " ".join(
                        [
                            insight.get("hook", ""),
                            insight.get("explanation", ""),
                            insight.get("action", ""),
                        ]
                    ),  # full insight
                    insight.get("hook", ""),  # hook
                    insight.get("explanation", ""),  # explanation
                    insight.get("action", ""),  # action
                    insight.get("source_name", ""),  # source name
                    insight.get("source_url", ""),  # source url
                    cohort.get("cohort_params", ""),  # cohort params
                    cohort.get("description", ""),  # cohort desc
                    template.get("type", ""),  # insight_template
                    metadata.get("generation_model", ""),  # generation model
                    metadata.get(
                        "generation_temperature", ""
                    ),  # generation temperature
                    metadata.get(
                        "generation_max_tokens", ""
                    ),  # generation max tokens
                    metadata.get(
                        "generation_timestamp", ""
                    ),  # generation timestamp
                    eval_result.get("overall_score", ""),  # eval overall score
                    eval_result.get("factual_accuracy", {}).get(
                        "score", ""
                    ),  # factual accuracy score
                    eval_result.get("safety", {}).get(
                        "score", ""
                    ),  # safety score
                    eval_result.get("source_faithfulness", {}).get(
                        "score", ""
                    ),  # faithfulness score
                    eval_result.get("relevance", {}).get(
                        "score", ""
                    ),  # relevance score
                    eval_result.get("actionability", {}).get(
                        "score", ""
                    ),  # actionability score
                    eval_result.get("cultural_appropriateness", {}).get(
                        "score", ""
                    ),  # culture appropriateness score
                    eval_result.get("recommendation", ""),  # recommendation
                    eval_result.get(
                        "revision_suggestions", ""
                    ),  # revision suggestions
                    eval.get("evaluation_model", ""),  # eval model
                    eval.get("evaluation_timestamp", ""),  # eval timestamp
                ]
            )

    print(f"\n✓ Exported {len(insights)} insights to CSV")
except Exception as e:
    print(f"Error exporting to CSV: {e}")


✓ Exported 410 insights to CSV


In [74]:
gemini = pd.read_csv("../output/gemini_flash_2.5/gemini_insights_final.csv")
deepseek = pd.read_csv("../output/deepseek/deepseek_insights_final.csv")
grok = pd.read_csv("../output/grok-fast/grok_insights_final.csv")

In [78]:
gemini["recommendation"].value_counts(normalize=True)

recommendation
revise     0.915094
approve    0.082547
reject     0.002358
Name: proportion, dtype: float64

In [77]:
deepseek["recommendation"].value_counts(normalize=True)

recommendation
revise     0.933985
approve    0.058680
reject     0.007335
Name: proportion, dtype: float64

In [76]:
grok["recommendation"].value_counts(normalize=True)

recommendation
revise     0.928571
approve    0.068027
reject     0.003401
Name: proportion, dtype: float64

In [55]:
metrics = ["eval_score", "factual_accuracy", "safety", "faithfulness",
           "relevance", "actionability", "cultural_appropriateness"]

In [63]:
gemini["cohort_desc"]

0      Sedentary office workers at risk of metabolic ...
1      Sedentary office workers at risk of metabolic ...
2      Sedentary office workers at risk of metabolic ...
3      Sedentary office workers at risk of metabolic ...
4      Sedentary office workers at risk of metabolic ...
                             ...                        
420           Women navigating perimenopausal transition
421           Women navigating perimenopausal transition
422           Women navigating perimenopausal transition
423           Women navigating perimenopausal transition
424           Women navigating perimenopausal transition
Name: cohort_desc, Length: 425, dtype: object

In [71]:
df = grok

In [72]:
for cohort_desc in df["cohort_desc"].unique():
    print("="*80)
    print(cohort_desc)
    for template in df["insight_template"].unique():
        print("-"*80)
        print(template)
        print(df[(df["insight_template"] == template) &\
               (df["cohort_desc"] == cohort_desc)]["hook"].values)
        print("-"*80)
    print("="*80)

Sedentary office workers at risk of metabolic diseases
--------------------------------------------------------------------------------
quantified_action_benefit
['Did you know a 10-minute brisk walk after lunch reduces post-meal blood glucose by 22%?'
 'Did you know standing for 3 minutes every 30 minutes of sitting improves insulin levels by 32%?'
 'Did you know 30g daily nuts reduce heart disease risk by 29% in at-risk adults?'
 'Did you know ordering hawker veggie sides boosting fiber by 10g daily cuts diabetes risk by 19%?'
 'Did you know 150 minutes weekend moderate activity halves cardiovascular disease risk despite weekday sedentary?']
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
comparative_effectiveness
['Did you know 10-minute walks after meals lower blood sugar spikes 22% more than one 30-minute daily walk?'
 'Did you know eating hawker veggies before rice cu

In [70]:
for cohort_desc in gemini["cohort_desc"].unique():
    print("="*80)
    print(cohort_desc)
    for template in gemini["insight_template"].unique():
        print("-"*80)
        print(template)
        print(gemini[(gemini["insight_template"] == template) &\
               (gemini["cohort_desc"] == cohort_desc)]["hook"].values)
        print("-"*80)
    print("="*80)

Sedentary office workers at risk of metabolic diseases
--------------------------------------------------------------------------------
quantified_action_benefit
['Did you know prolonged sitting increases your risk of premature death by up to 50%?'
 'Did you know replacing one sugary drink daily with water can reduce your risk of type 2 diabetes by 18%?'
 'Did you know taking a 15-minute brisk walk after lunch can reduce post-meal blood sugar spikes by 20-30%?'
 'Did you know eating one handful (30g) of nuts daily can reduce your risk of heart disease by 30%?'
 'Did you know achieving 7-8 hours of quality sleep nightly can lower your risk of obesity by 50%?']
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
comparative_effectiveness
['Did you know 10-minute walk after each meal lowers blood sugar 12% more than one 30-minute daily walk?'
 'Did you know standing for just 5 min

In [134]:
summarized_insights.to_excel("./gemini.xlsx")

In [146]:
output_dir = "../output/gemini_flash_2.5"

In [169]:
eval_result.get("culture_appropriateness", {}).get("score", "")            

        

''

In [179]:
loader = ConfigLoader("singapore")

In [18]:
loader.priority_cohorts

[{'name': 'prediabetic_office_workers',
  'dimensions': {'age_group': ['30-44', '45-59'],
   'bmi': ['overweight', 'obese'],
   'physical_activity': ['sedentary'],
   'occupation_context': ['desk_job']},
  'priority': 1,
  'description': 'Sedentary office workers at high diabetes risk',
  'rationale': "Singapore's 8.6% diabetes rate driven by sedentary work culture + weight. Sitting 8+ hrs daily compounds metabolic risk. High prevention potential through workplace interventions.",
  'insight_angles': ['Desk-based micro-movements (standing breaks, walking meetings)',
   'Hawker center meal optimization for busy workers',
   'Post-lunch walks to manage blood sugar spikes',
   'Weekend activity catch-up strategies']},
 {'name': 'shift_workers_metabolic_risk',
  'dimensions': {'age_group': ['30-44', '45-59'],
   'occupation_context': ['shift_work'],
   'health_conditions': ['prediabetes', 'hypertension']},
  'priority': 1,
  'description': 'Shift workers with disrupted circadian rhythms',


In [13]:
generator = CohortGenerator(market="singapore")

In [14]:
generator.generate_priority_cohorts()

[{'cohort_id': 'cohort_0005',
  'cohort_params': {'age_group': '30-44',
   'occupation_context': 'shift_work',
   'health_conditions': 'prediabetes'},
  'priority_level': 1,
  'description': '30-44 years old, shift_work, prediabetes'},
 {'cohort_id': 'cohort_0007',
  'cohort_params': {'age_group': '45-59',
   'occupation_context': 'shift_work',
   'health_conditions': 'prediabetes'},
  'priority_level': 1,
  'description': '45-59 years old, shift_work, prediabetes'},
 {'cohort_id': 'cohort_0028',
  'cohort_params': {'age_group': '18-29',
   'occupation_context': 'student',
   'health_conditions': 'stress'},
  'priority_level': 1,
  'description': '18-29 years old, student, stress'},
 {'cohort_id': 'cohort_0029',
  'cohort_params': {'age_group': '18-29',
   'occupation_context': 'student',
   'health_conditions': 'anxiety'},
  'priority_level': 1,
  'description': '18-29 years old, student, anxiety'},
 {'cohort_id': 'cohort_0030',
  'cohort_params': {'age_group': '18-29',
   'occupation

In [49]:
import pandas as pd
test = pd.read_json("../output/insights_raw.json")

In [59]:
for insight in [" ".join([v["action"]]) for v in test["insights"].values]:
    print(insight)

Join the National Steps Challenge app today, aim for 10,000 daily steps like your peers to enhance mobility.
Start HPB's free online strength workouts twice weekly, like squats and resistance bands, to build muscle with others.
Download the HPB National Steps Challenge app, aim for 10,000 steps daily, and track progress with friends for motivation.
Register for HPB's free Live Healthy! weight management sessions on HealthHub.sg and start sustainable changes today.
Download HealthHub app, register for National Steps Challenge, and apply SPF30+ sunscreen with hat before stepping out.
Log into HealthHub.sg, use HPB weight management tools to track BMI, diet, and activity for improved skin health.
Lose 1kg this month: swap one sugary drink for water daily and track 30-min brisk walks with HPB's Healthy 365 app.
Perform 20 mins of squats, lunges, planks twice weekly; attend free ActiveSG strength classes at community centres.
Walk briskly for 30 minutes daily, e.g., to MRT, and cut salt by 

In [41]:
response = requests.head("https://www.hpb.gov.sg/healthy-living/healthy-360/national-steps-challenge", timeout=5, allow_redirects=True)

In [42]:
response.raise_for_status()

In [43]:
response

<Response [200]>

In [2]:
loader = ConfigLoader(market="singapore")

In [3]:
domains = loader.health_domains
templates = loader.insight_templates

In [5]:
templates

{'quantified_action_benefit': {'type': 'quantified_action_benefit',
  'description': 'Specific action with precise time/frequency that produces measurable outcome',
  'weight': 15,
  'structure': '[Action] for [duration/frequency] [produces dramatic result with %].\n[Brief mechanism why]. [Immediate specific action to start today].\n',
  'example': ["150 minutes moderate activity weekly reduces chronic disease by 30%—that's only 21 minutes daily. Start with a brisk walk during lunch break today.",
   'Sleeping 7-8 hours nightly has lowest mortality risk. This sweet spot optimizes immune function, metabolism, cellular repair. Prioritize consistent 7-8 hour sleep windows.',
   'Eating 1 handful (30g) nuts daily reduces heart disease by 30%, stroke by 20%, all-cause mortality by 22%. Include almonds, walnuts, pistachios daily.'],
  'tone': 'Direct, evidence-driven, immediately actionable'},
 'comparative_effectiveness': {'type': 'comparative_effectiveness',
  'description': 'Compare two a

In [63]:
list(domains.keys())

['cardiovascular',
 'metabolic',
 'respiratory',
 'sleep',
 'musculoskeletal',
 'bone_health',
 'mental_health',
 'cognitive_health',
 'nutrition',
 'physical_activity',
 'preventive_care',
 'cancer_prevention',
 'screening',
 'digestive_health',
 'kidney_health',
 'sexual_reproductive_health',
 'skin_health',
 'oral_health',
 'sensory_health',
 'immune_health']

In [14]:
loader.sample_health_domains()

[{'name': 'bone-health',
  'subcategories': ['osteoporosis',
   'bone-density',
   'fracture-prevention',
   'calcium-vitamin-d']}]

In [29]:
sources = loader.sources

In [36]:
[src["url"] for src in sources["tier_1_local"]["sources"]]

['https://www.healthhub.sg',
 'https://www.moh.gov.sg',
 'https://www.nhg.com.sg',
 'https://www.singhealth.com.sg',
 'https://www.nuhs.edu.sg',
 'https://www.moh.gov.sg/resources-statistics/reports/national-population-health-survey-2024',
 'https://nus.edu.sg',
 'https://www.duke-nus.edu.sg',
 'https://www.ntu.edu.sg/medicine',
 'https://www.ace-hta.gov.sg',
 'https://www.nrdo.gov.sg',
 'https://www.aic.sg',
 'https://www.hsa.gov.sg']

In [33]:
[sources[tier][url] for tier in sources for url in sources[tier]]

['Singapore government and healthcare institutions - highest priority',
 [{'name': 'Health Promotion Board (HPB)',
   'url': 'https://www.healthhub.sg',
   'focus': 'National health promotion, lifestyle guidelines, screening programs'},
  {'name': 'Ministry of Health Singapore (MOH)',
   'url': 'https://www.moh.gov.sg',
   'focus': 'National health policies, disease statistics, clinical guidelines'},
  {'name': 'National Healthcare Group (NHG)',
   'url': 'https://www.nhg.com.sg',
   'focus': 'Clinical research, primary or specialty care, population health'},
  {'name': 'SingHealth',
   'url': 'https://www.singhealth.com.sg',
   'focus': 'Clinical research, primary or specialty care, population health'},
  {'name': 'National University Health Group (NUHS)',
   'url': 'https://www.nuhs.edu.sg',
   'focus': 'Clinical research, primary or specialty care, population health'},
  {'name': 'Singapore NPHS (National Population Health Survey)',
   'url': 'https://www.moh.gov.sg/resources-statis

In [1]:
test = {
      "hook": "Did you know regular exercise + balanced diet doubles protection against depression compared to either alone?",
      "explanation": "For obese Singaporeans aged 50-59 at higher depression risk from inflammation and low mood, this synergy boosts brain chemicals like serotonin more effectively, enhancing emotional wellbeing as promoted by HPB guidelines (40 words)",
      "action": "Aim for 150min weekly brisk walking and follow HPB My Healthy Plate: fill half your plate with fruits/veggies, quarter wholegrains, quarter lean protein.",
      "source_name": "Health Promotion Board (HPB)",
      "source_url": "https://www.hpb.gov.sg/healthy-living/mental-wellbeing",
      "numeric_claim": "doubles protection"
    }

In [7]:
from urllib.parse import urlparse

In [15]:
source_url = "https://www.who.int/news-room/fact-sheets/detail/physical-activity"
parsed = urlparse(source_url)

In [11]:
source_url = "stupid idiot"
parsed = urlparse(source_url)

In [12]:
parsed

ParseResult(scheme='', netloc='', path='stupid idiot', params='', query='', fragment='')

In [18]:
import requests

In [25]:
try:
    response = requests.head("https://tada", timeout=5)
except:
    response.raise_for_status()

In [26]:
sources

NameError: name 'sources' is not defined

In [ ]:
if not parsed.scheme or not parsed.netloc:
    print("Invalid URL")

In [3]:
required_fields = [
            "hook",
            "explanation",
            "action",
            "source_name",
            "source_url",
            "numeric_claim",
        ]

In [ ]:
if __name__ == "__main__":
    # Example usage
    prompt_template = PromptTemplates()

    


In [32]:
tier_to_names = {
    tier: [src["name"] for src in info["sources"]]
    for tier, info in sources.items()
}


In [33]:
tier_to_names

{'tier_1_local': ['Health Promotion Board (HPB)',
  'Ministry of Health Singapore (MOH)',
  'National Healthcare Group (NHG)',
  'SingHealth',
  'National University Health Group (NUHS)',
  'Singapore NPHS (National Population Health Survey)',
  'National University of Singapore',
  'Duke-NUS',
  'LKC Medicine',
  'Agency for Care Effectiveness (ACE)',
  'National Registry of Diseases Office (NRDO)',
  'Agency for Integrated Care (AIC)',
  'Health Sciences Authority (HSA)'],
 'tier_2_global': ['World Health Organization (WHO)',
  'World Bank Health Data',
  'UNICEF',
  'OECD Health Statistics',
  'UK National Institute for Health and Care Excellence (NICE)',
  'US Centers for Disease Control (CDC)',
  'The Lancet',
  'JAMA (Journal of American Medical Association)',
  'New England Journal of Medicine (NEJM)',
  'BMJ (British Medical Journal)',
  'Nature Medicine & Science'],
 'tier_3_research': ['PubMed/MEDLINE',
  'Cochrane Library',
  'NIH (National Institutes of Health)']}

In [30]:
[d["name"] for d in sources["tier_1_local"]["sources"]]

['Health Promotion Board (HPB)',
 'Ministry of Health Singapore (MOH)',
 'National Healthcare Group (NHG)',
 'SingHealth',
 'National University Health Group (NUHS)',
 'Singapore NPHS (National Population Health Survey)',
 'National University of Singapore',
 'Duke-NUS',
 'LKC Medicine',
 'Agency for Care Effectiveness (ACE)',
 'National Registry of Diseases Office (NRDO)',
 'Agency for Integrated Care (AIC)',
 'Health Sciences Authority (HSA)']

In [18]:
generator = CohortGenerator()

In [19]:
generator.generate_priority_cohorts()

[{'cohort_id': 'cohort_0001',
  'cohort_params': {'age_group': '30-39', 'bmi': 'overweight'},
  'priority_level': 2,
  'description': '30-39 years old, overweight'},
 {'cohort_id': 'cohort_0002',
  'cohort_params': {'age_group': '30-39', 'bmi': 'obese'},
  'priority_level': 2,
  'description': '30-39 years old, obese'},
 {'cohort_id': 'cohort_0003',
  'cohort_params': {'age_group': '40-49', 'bmi': 'overweight'},
  'priority_level': 2,
  'description': '40-49 years old, overweight'},
 {'cohort_id': 'cohort_0004',
  'cohort_params': {'age_group': '40-49', 'bmi': 'obese'},
  'priority_level': 2,
  'description': '40-49 years old, obese'},
 {'cohort_id': 'cohort_0005',
  'cohort_params': {'age_group': '50-59', 'bmi': 'overweight'},
  'priority_level': 2,
  'description': '50-59 years old, overweight'},
 {'cohort_id': 'cohort_0006',
  'cohort_params': {'age_group': '50-59', 'bmi': 'obese'},
  'priority_level': 2,
  'description': '50-59 years old, obese'},
 {'cohort_id': 'cohort_0007',
  'c

In [16]:
domains

{'cardiovascular': {'name': 'cardiovascular',
  'subcategories': ['heart-health',
   'blood-pressure',
   'cholesterol',
   'circulation']},
 'metabolic': {'name': 'metabolic',
  'subcategories': ['diabetes',
   'weight-management',
   'thyroid',
   'fatty-liver']},
 'respiratory': {'name': 'respiratory',
  'subcategories': ['lung-health', 'asthma', 'copd', 'air-quality']},
 'sleep': {'name': 'sleep',
  'subcategories': ['sleep-quality', 'sleep-duration', 'sleep-disorders']},
 'musculoskeletal': {'name': 'musculoskeletal',
  'subcategories': ['joint-health',
   'muscle-health',
   'mobility',
   'pain-management']},
 'bone_health': {'name': 'bone-health',
  'subcategories': ['osteoporosis',
   'bone-density',
   'fracture-prevention',
   'calcium-vitamin-d']},
 'mental_health': {'name': 'mental-health',
  'subcategories': ['stress-management',
   'depression',
   'anxiety',
   'emotional-wellbeing']},
 'cognitive_health': {'name': 'cognitive-health',
  'subcategories': ['brain-health',

In [ ]:
import random

keys = list(template.keys())
weights = [templates[k][""]]

{'risk_amplification': {'type': 'risk_amplification',
  'description': 'Highlight significantly elevated risks for specific cohorts',
  'weight': 2,
  'example': 'Men aged 40-49 with diabetes have 3x higher risk of heart disease',
  'tone': 'Factual, non-alarmist'},
 'optimization_strategies': {'type': 'optimization_strategies',
  'description': 'Actionable ways to maximize health benefits and protective factors',
  'weight': 10,
  'example': 'Adults aged 50+ who walk 30 min daily reduce osteoporosis risk by 40%',
  'tone': 'Empowering, action-oriented'},
 'protective_synergies': {'type': 'protective_synergies',
  'description': 'Combined protective effects of multiple positive behaviors',
  'weight': 5,
  'example': 'Non-smoking + 150min exercise weekly creates 5x stronger heart protection than either alone',
  'tone': 'Encouraging, synergistic'},
 'early_detection': {'type': 'early_detection',
  'description': 'Screening and early intervention opportunities',
  'weight': 7,
  'exampl

In [ ]:
class PromptTemplates:
    """Collection of prompt templates for different generation modes."""

    def pure_llm_generation(
        cohort_description: str,
        cohort_params: dict,
        template_params: str,
        domain_params: str,
        region: str = "singapore",
    ) -> str:
        """
        Generate prompt for pure LLM-based insight generation (no external tools).
        Uses LLM's pre-trained knowledge only.
        """

        prompt = f"""
        You are a medical and public health expert generating evidence-based health insights for a health application.

        REGION:
        - {region}

        TARGET COHORT:
        {cohort_description}

        Cohort Parameters: {cohort_params}

        INSIGHT TEMPLATE SELECTED:
        - Type: {template_params["type"]}
        - Description: {template_params["description"]}
        - Required Tone: "{template_params["tone"]}"
        - Example Pattern: "{template_params["example"]}"

        HEALTH DOMAIN:
        - Domain: {health_domain}
        - Domain Description: {domain_meta.get("description", "")}
        - Relevant Conditions: {domain_meta.get("conditions", [])}

        AUTHORITATIVE SOURCES FOR {region.upper()}:
        {region_meta["top_sources"]}

        TASK:
        Generate a "Did You Know" (DYK) health insight specifically tailored to this cohort. The insight should be:
        1. Evidence-based and scientifically accurate
        2. Highly relevant to this specific demographic and health domain
        3. Actionable and motivating
        4. Culturally appropriate for {region}
        5. Following the conceptual intent of the selected template (“{template_meta["description"]}”)

        OUTPUT FORMAT (JSON):
        {{
        "hook": "A compelling, attention-grabbing fact that starts with 'Did you know...' (max 20 words)",
        "explanation": "A brief, evidence-based explanation of why this matters for this specific cohort (40-60 words)",
        "action": "A specific, actionable step the user can take (20-30 words)",
        "source_name": "Name of the authoritative source (e.g., WHO, CDC, HPB, peer-reviewed journal)",
        "source_url": "URL if available, or 'general medical knowledge'",
        "numeric_claim": "Any specific numeric claim made (e.g., '3x higher risk', '30% reduction')",
        "confidence": "high/medium/low - your confidence in this claim"
        }}

        IMPORTANT REQUIREMENTS:
        - Be specific to the cohort's characteristics (age, gender, lifestyle, conditions)
        - Use actual statistics when possible (but be accurate - don't make up numbers)
        - Cite reputable sources
        - Keep language clear and motivating
        - Ensure the action is practical, achievable, region-appropriate and cohort-specific

        DO NOT:
        - Make up statistics or data
        - Use vague or generic insights that could apply to anyone
        - Include medical advice that requires professional diagnosis
        - Use fear-mongering language

        Return ONLY valid JSON, no additional text.
        """

        return prompt

In [35]:
prompttemplate = PromptTemplates()

In [41]:
templates["myth_busting"]

{'description': 'Counter-intuitive facts that challenge common misconceptions',
 'weight': 5,
 'example': 'Myth: Low-fat diets prevent heart disease. Reality: Quality fats (omega-3) protect hearts',
 'tone': 'Surprising, evidence-based'}

In [36]:
cohort = {
      "cohort_id": "cohort_0001",
      "cohort_params": {
        "age_group": "30-39",
        "bmi": "overweight"
      },
      "priority_level": 2,
      "description": "30-39 years old, overweight"
    }

In [38]:
domain

{'cardiovascular': {'subcategories': ['heart-health',
   'blood-pressure',
   'cholesterol',
   'circulation']},
 'metabolic': {'subcategories': ['diabetes',
   'weight-management',
   'thyroid',
   'fatty-liver']},
 'respiratory': {'subcategories': ['lung-health',
   'asthma',
   'copd',
   'air-quality']},
 'sleep': {'subcategories': ['sleep-quality',
   'sleep-duration',
   'sleep-disorders']},
 'musculoskeletal': {'subcategories': ['joint-health',
   'muscle-health',
   'mobility',
   'pain-management']},
 'bone-health': {'subcategories': ['osteoporosis',
   'bone-density',
   'fracture-prevention',
   'calcium-vitamin-d']},
 'mental-health': {'subcategories': ['stress-management',
   'depression',
   'anxiety',
   'emotional-wellbeing']},
 'cognitive-health': {'subcategories': ['brain-health',
   'memory',
   'dementia-prevention',
   'mental-sharpness']},
 'nutrition': {'subcategories': ['healthy-eating',
   'dietary-patterns',
   'hydration',
   'supplements']},
 'physical-activ

In [39]:
prompttemplate.pure_llm_generation(cohort_description=cohort["description"],
                                   cohort_params=cohort["cohort_params"],
                                   template_type=templates["myth_busting"],
                                   health_domain=domain["cardiovascular"],
                                   region="singapore")

TypeError: PromptTemplates.pure_llm_generation() got multiple values for argument 'cohort_description'

In [33]:
templates

{'risk_amplification': {'description': 'Highlight significantly elevated risks for specific cohorts',
  'weight': 2,
  'example': 'Men aged 40-49 with diabetes have 3x higher risk of heart disease',
  'tone': 'Factual, non-alarmist'},
 'optimization_strategies': {'description': 'Actionable ways to maximize health benefits and protective factors',
  'weight': 10,
  'example': 'Adults aged 50+ who walk 30 min daily reduce osteoporosis risk by 40%',
  'tone': 'Empowering, action-oriented'},
 'protective_synergies': {'description': 'Combined protective effects of multiple positive behaviors',
  'weight': 5,
  'example': 'Non-smoking + 150min exercise weekly creates 5x stronger heart protection than either alone',
  'tone': 'Encouraging, synergistic'},
 'early_detection': {'description': 'Screening and early intervention opportunities',
  'weight': 7,
  'example': 'Annual HbA1c screening catches prediabetes 5 years before diabetes symptoms appear',
  'tone': 'Proactive, preventive'},
 'temp

In [32]:
loader.sources["evidence_sources"]["tier_1_local"]["sources"]

[{'name': 'Health Promotion Board (HPB)',
  'url': 'https://www.healthhub.sg',
  'focus': 'National health promotion, lifestyle guidelines, screening programs'},
 {'name': 'Ministry of Health Singapore (MOH)',
  'url': 'https://www.moh.gov.sg',
  'focus': 'National health policies, disease statistics, clinical guidelines'},
 {'name': 'National Healthcare Group (NHG)',
  'url': 'https://www.nhg.com.sg',
  'focus': 'Clinical research, primary or specialty care, population health'},
 {'name': 'SingHealth',
  'url': 'https://www.singhealth.com.sg',
  'focus': 'Clinical research, primary or specialty care, population health'},
 {'name': 'National University Health Group (NUHS)',
  'url': 'https://www.nuhs.edu.sg',
  'focus': 'Clinical research, primary or specialty care, population health'},
 {'name': 'Singapore NPHS (National Population Health Survey)',
  'url': 'https://www.moh.gov.sg/resources-statistics/reports/national-population-health-survey-2024',
  'focus': 'Population health data,

In [12]:
for tier, v in loader.sources["evidence_sources"].items():
    print(tier, v["sources"])
    print([source["name"] for source in v["sources"]])
    break

tier_1_local [{'name': 'Health Promotion Board (HPB)', 'url': 'https://www.healthhub.sg', 'focus': 'National health promotion, lifestyle guidelines, screening programs'}, {'name': 'Ministry of Health Singapore (MOH)', 'url': 'https://www.moh.gov.sg', 'focus': 'National health policies, disease statistics, clinical guidelines'}, {'name': 'National Healthcare Group (NHG)', 'url': 'https://www.nhg.com.sg', 'focus': 'Clinical research, primary or specialty care, population health'}, {'name': 'SingHealth', 'url': 'https://www.singhealth.com.sg', 'focus': 'Clinical research, primary or specialty care, population health'}, {'name': 'National University Health Group (NUHS)', 'url': 'https://www.nuhs.edu.sg', 'focus': 'Clinical research, primary or specialty care, population health'}, {'name': 'Singapore NPHS (National Population Health Survey)', 'url': 'https://www.moh.gov.sg/resources-statistics/reports/national-population-health-survey-2024', 'focus': 'Population health data, disease prevale

In [29]:
templates = loader.insight_templates["insight_templates"]

In [31]:
len(templates)

17

In [24]:
domain = loader.health_domains["health_domains"]

In [26]:
domain["cardiovascular"]

{'subcategories': ['heart-health',
  'blood-pressure',
  'cholesterol',
  'circulation']}

In [ ]:
for template in templates:
    print(template["type"])
    break

[{'type': 'risk_amplification',
  'description': 'Highlight significantly elevated risks for specific cohorts',
  'weight': 2,
  'example': 'Men aged 40-49 with diabetes have 3x higher risk of heart disease',
  'tone': 'Factual, non-alarmist'},
 {'type': 'optimization_strategies',
  'description': 'Actionable ways to maximize health benefits and protective factors',
  'weight': 10,
  'example': 'Adults aged 50+ who walk 30 min daily reduce osteoporosis risk by 40%',
  'tone': 'Empowering, action-oriented'},
 {'type': 'protective_synergies',
  'description': 'Combined protective effects of multiple positive behaviors',
  'weight': 5,
  'example': 'Non-smoking + 150min exercise weekly creates 5x stronger heart protection than either alone',
  'tone': 'Encouraging, synergistic'},
 {'type': 'early_detection',
  'description': 'Screening and early intervention opportunities',
  'weight': 7,
  'example': 'Annual HbA1c screening catches prediabetes 5 years before diabetes symptoms appear',
  

In [2]:
# define top-level variables
MARKET = "singapore"

# generating config via loader
loader = ConfigLoader()
config = loader.load_market_config(MARKET)

# generate generator
generator = CohortGenerator(market=MARKET)

In [3]:
generator.generate_all_single_dimension_cohorts()

[{'cohort_id': 'cohort_10000',
  'cohort_params': {'age_group': '0-17'},
  'priority_level': 1,
  'description': '0-17'},
 {'cohort_id': 'cohort_10001',
  'cohort_params': {'age_group': '18-29'},
  'priority_level': 1,
  'description': '18-29'},
 {'cohort_id': 'cohort_10002',
  'cohort_params': {'age_group': '30-39'},
  'priority_level': 1,
  'description': '30-39'},
 {'cohort_id': 'cohort_10003',
  'cohort_params': {'age_group': '40-49'},
  'priority_level': 1,
  'description': '40-49'},
 {'cohort_id': 'cohort_10004',
  'cohort_params': {'age_group': '50-59'},
  'priority_level': 1,
  'description': '50-59'},
 {'cohort_id': 'cohort_10005',
  'cohort_params': {'age_group': '60-69'},
  'priority_level': 1,
  'description': '60-69'},
 {'cohort_id': 'cohort_10006',
  'cohort_params': {'age_group': '70+'},
  'priority_level': 2,
  'description': '70+'},
 {'cohort_id': 'cohort_10007',
  'cohort_params': {'gender': 'male'},
  'priority_level': 1,
  'description': 'male'},
 {'cohort_id': 'coh

In [13]:
# generate priority cohorts
cohorts = []
cohort_id = 1

for priority_group in priority_cohorts: # for each priority group
    dimensions = priority_group["dimensions"]
    dimension_names = list(dimensions.keys())
    dimension_values = [dimensions[dim] for dim in dimension_names]

    for combination in product(*dimension_values):
        cohort_spec = {
            "cohort_id": f"cohort_{cohort_id:04d}",
            "cohort_params": {},
            "priority_level": generator._calculate_priority(
                dict(zip(dimension_names, combination))
            ),
        }
        
        # Build cohort parameters
        for dim_name, value in zip(dimension_names, combination):
            cohort_spec["cohort_params"][dim_name] = value

        # Add human-readable description
        cohort_spec["description"] = generator._generate_description(
            cohort_spec["cohort_params"]
        )

        cohorts.append(cohort_spec)
        cohort_id += 1
        
# Sort by priority (lower number = higher priority)
cohorts.sort(key=lambda x: x["priority_level"])

In [ ]:


    def generate_all_single_dimension_cohorts(self) -> List[Dict[str, Any]]:
        """
        Generate cohorts for each single dimension (fallback strategy).
        Useful for ensuring basic coverage.
        """
        cohorts = []
        cohort_id = 10000  # Start from higher ID to distinguish

        for dimension_name, values in self.cohort_definitions.items():
            for value_spec in values:
                value = value_spec["name"]
                cohort_spec = {
                    "cohort_id": f"cohort_{cohort_id:04d}",
                    "cohort_params": {dimension_name: value},
                    "priority_level": value_spec.get("priority", 5),
                    "description": f"{value.replace('-', ' ')}",
                }
                cohorts.append(cohort_spec)
                cohort_id += 1

        return cohorts

    def export_cohorts(self, output_path: str = "cohorts.json"):
        """Export generated cohorts to JSON file."""
        priority_cohorts = self.generate_priority_cohorts()
        single_cohorts = self.generate_all_single_dimension_cohorts()

        all_cohorts = {
            "priority_cohorts": priority_cohorts,
            "single_dimension_cohorts": single_cohorts,
            "total_priority": len(priority_cohorts),
            "total_single": len(single_cohorts),
            "generation_strategy": "priority_based",
        }

        with open(output_path, "w") as f:
            json.dump(all_cohorts, f, indent=2)

        print(f"Generated {len(priority_cohorts)} priority cohorts")
        print(f"Generated {len(single_cohorts)} single-dimension cohorts")
        print(f"Total: {len(priority_cohorts) + len(single_cohorts)} cohorts")
        print(f"Exported to {output_path}")

        return all_cohorts

    def get_cohort_statistics(self) -> Dict[str, Any]:
        """Get statistics about generated cohorts."""
        priority_cohorts = self.generate_priority_cohorts()

        stats = {
            "total_cohorts": len(priority_cohorts),
            "by_priority": defaultdict(int),
            "dimensions_used": defaultdict(int),
        }

        for cohort in priority_cohorts:
            stats["by_priority"][cohort["priority_level"]] += 1
            for dimension in cohort["cohort_params"].keys():
                stats["dimensions_used"][dimension] += 1

        return dict(stats)

# Class walkthrough

In [2]:
# configloader object
configloader = ConfigLoader()

# region-specific config
# cohort definitions, health domains, insight templates
config = configloader.load_market_config("singapore")

# sorted insight templates
sorted_templates = configloader.get_templates_by_weight(config) 

# get specific insight template
opt_strategies = configloader.get_template_by_type(config,
                                                   template_type="optimization_strategies")

In [3]:
cohortgenerator = CohortGenerator(market="singapore", config=config)

In [4]:
cohortgenerator.cohort_definitions.keys()

dict_keys(['age_group', 'gender', 'race', 'smoking_status', 'physical_activity', 'bmi_category', 'chronic_conditions'])

In [5]:
cohortgenerator.get_cohort_statistics()

{'total_cohorts': 38,
 'by_priority': defaultdict(int, {1: 1, 2: 28, 3: 9}),
 'dimensions_used': defaultdict(int,
             {'age_group': 38,
              'chronic_conditions': 14,
              'smoking_status': 4,
              'physical_activity': 4,
              'bmi_category': 6,
              'gender': 16,
              'race': 4})}

In [6]:
cohortgenerator.generate_all_single_dimension_cohorts()

[{'cohort_id': 'cohort_10000',
  'cohort_params': {'age_group': '0-17'},
  'priority_level': 1,
  'description': '0 17'},
 {'cohort_id': 'cohort_10001',
  'cohort_params': {'age_group': '18-29'},
  'priority_level': 1,
  'description': '18 29'},
 {'cohort_id': 'cohort_10002',
  'cohort_params': {'age_group': '30-39'},
  'priority_level': 1,
  'description': '30 39'},
 {'cohort_id': 'cohort_10003',
  'cohort_params': {'age_group': '40-49'},
  'priority_level': 1,
  'description': '40 49'},
 {'cohort_id': 'cohort_10004',
  'cohort_params': {'age_group': '50-59'},
  'priority_level': 1,
  'description': '50 59'},
 {'cohort_id': 'cohort_10005',
  'cohort_params': {'age_group': '60-69'},
  'priority_level': 1,
  'description': '60 69'},
 {'cohort_id': 'cohort_10006',
  'cohort_params': {'age_group': '70+'},
  'priority_level': 2,
  'description': '70+'},
 {'cohort_id': 'cohort_10007',
  'cohort_params': {'gender': 'male'},
  'priority_level': 1,
  'description': 'male'},
 {'cohort_id': 'coh